In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [3]:
# Replace 'path_to_csv' with the actual path to your CSV file
data = pd.read_csv(r'C:\Users\manfr\OneDrive\Escritorio\Manfred\EMAI\SecondSemester_Slovenia\Project\BSNIP_data\BSNIP_data\neural\neural_parcellated_gbc.csv')

data = data.drop('Unnamed: 0', axis=1)
data['binary_target'] = (data['Group'] == 'CON').astype(int)
unique_value_counts_binary = data['binary_target'].value_counts()
unique_value_counts_multiclass = data['Group'].value_counts()
print(unique_value_counts_binary)
print(unique_value_counts_multiclass)
# Create a list of the new column order
columns = ['binary_target', 'Group'] + [col for col in data.columns if col not in ['binary_target', 'Group']]

# Reindex the DataFrame with the new column order
data = data[columns]
data.head()

binary_target
0    436
1    202
Name: count, dtype: int64
Group
CON     202
SCZP    167
BPP     150
SADP    119
Name: count, dtype: int64


,binary_target,Group,X1,X2,X3,X4,X5,X6,X7,X8,...,X709,X710,X711,X712,X713,X714,X715,X716,X717,X718
0,0,SADP,0.017901,0.009985,0.003740,0.012205,0.005254,0.023362,0.009241,-0.013089,...,0.014800,-0.000650,-0.007422,-0.003942,0.005397,0.000533,0.013678,-0.014425,0.000760,0.000889
1,0,SCZP,0.006281,0.014014,-0.005585,-0.000536,0.003254,0.008885,0.008263,-0.007209,...,-0.002154,0.007586,0.004824,0.008781,-0.007612,0.005459,0.004697,0.002705,0.008613,0.005739
2,0,BPP,-0.011590,0.009900,-0.008592,-0.009722,0.001935,0.016690,0.007919,-0.006034,...,-0.003193,-0.018124,0.012447,-0.000854,-0.003363,0.001340,-0.007613,0.001474,-0.003668,0.005865
3,0,SCZP,0.017341,0.009125,0.001433,0.005714,0.000891,0.004737,0.000913,-0.004097,...,-0.006566,0.018083,0.007969,0.005143,0.010550,0.000927,0.015112,0.003535,0.008583,0.006735
4,0,SADP,-0.025011,0.031992,-0.037631,-0.024702,-0.009084,-0.002958,0.031740,0.018152,...,-0.004469,0.037608,0.032018,0.005771,0.000471,0.017834,0.020844,0.039692,0.029756,0.003653


## Binary Classification

In [4]:
seed = 10
binary_data = data.drop('Group', axis=1)
input_features = binary_data.drop('binary_target', axis=1).values
targets = binary_data['binary_target'].values
X_train, X_test, y_train, y_test = train_test_split(input_features, targets, test_size=0.2, random_state=seed)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(510, 718) (128, 718) (510,) (128,)


In [5]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train, y_train)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=10)

In [7]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train)
xgb_test_predictions = xgb_clf.predict(X_test)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train)
rf_test_predictions = rf_clf.predict(X_test)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.984375
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.015625
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.9765625
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.0234375
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.010961886875314282
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.01337213275159097


## Multiclass classification

In [8]:
seed = 10
binary_data = data.drop('binary_target', axis=1)
input_features = binary_data.drop('Group', axis=1).values
# Initialize the LabelEncoder
le = LabelEncoder()
targets_numeric = le.fit_transform(binary_data['Group'])

X_train, X_test, y_train, y_test = train_test_split(input_features, targets_numeric, 
                                                    test_size=0.2, random_state=seed, 
                                                    stratify=targets_numeric)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

# Access the mapping of encoded labels back to original class names
label_mapping = dict(zip(le.transform(le.classes_), le.classes_))
for target_values, name in [(y_train,'y_train'),(y_test,'y_test')]:
    # Get unique values and their counts
    unique_values, counts = np.unique(target_values, return_counts=True)
    print(f'Target Values statistics of: {name}')
    # Display the unique values and their counts
    for value, count in zip(unique_values, counts):
        print(f"Value: {label_mapping[value]}, Count: {count}")

(510, 718) (128, 718) (510,) (128,)
Target Values statistics of: y_train
Value: BPP, Count: 120
Value: CON, Count: 161
Value: SADP, Count: 95
Value: SCZP, Count: 134
Target Values statistics of: y_test
Value: BPP, Count: 30
Value: CON, Count: 41
Value: SADP, Count: 24
Value: SCZP, Count: 33


### XG Boost and Random Forest

In [9]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train, y_train)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=10)

In [10]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train)
xgb_test_predictions = xgb_clf.predict(X_test)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train)
rf_test_predictions = rf_clf.predict(X_test)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.578125
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.421875
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.5859375
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.4140625
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.04365136062231838
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.043536510010075705


## Data augmentation techniques

In [11]:
#! pip install imbalanced-learn

   ---------------------------------------- 257.7/257.7 kB 1.4 MB/s eta 0:00:00


In [26]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [27]:
# Assuming X_train, y_train are your initial training data

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=10)
rf.fit(X_train, y_train)

# Select features based on importance
selector = SelectFromModel(rf, prefit=True, threshold='mean')  # Selects features with importance higher than the mean importance

# Transform training and testing sets
X_train_reduced = selector.transform(X_train)
X_test_reduced = selector.transform(X_test)

print(f"Original number of features: {X_train.shape[1]}")
print(f"Reduced number of features: {X_train_reduced.shape[1]}")

smote = SMOTE(random_state=10)
X_train_smote, y_train_smote = smote.fit_resample(X_train_reduced, y_train)

# Check the distribution after SMOTE
unique_smote, counts_smote = np.unique(y_train_smote, return_counts=True)
for value, count in zip(unique_smote, counts_smote):
    print(f"Value: {label_mapping[value]}, Count: {count}")

# Determine the desired number of samples for each class
desired_counts = 300
target_counts = {label: desired_counts for label in np.unique(y_train_smote)}

# Initialize RandomOverSampler with the target_counts as the sampling strategy
ros = RandomOverSampler(random_state=10, sampling_strategy=target_counts)

# Now, apply RandomOverSampler to adjust class distribution to have 300 samples per class
X_train_augmented, y_train_augmented = ros.fit_resample(X_train_smote, y_train_smote)

# Verify the final distribution
unique_augmented, counts_augmented = np.unique(y_train_augmented, return_counts=True)
print('After augmentation to reach 300 samples per class:')
for value, count in zip(unique_augmented, counts_augmented):
    print(f"Value: {label_mapping[value]}, Count: {count}")

Original number of features: 718
Reduced number of features: 171
Value: BPP, Count: 161
Value: CON, Count: 161
Value: SADP, Count: 161
Value: SCZP, Count: 161
After augmentation to reach 300 samples per class:
Value: BPP, Count: 300
Value: CON, Count: 300
Value: SADP, Count: 300
Value: SCZP, Count: 300


In [28]:
# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', n_estimators=100, seed=seed)

# Fit the model
xgb_clf.fit(X_train_augmented, y_train_augmented)


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=seed)

# Fit the model
rf_clf.fit(X_train_augmented, y_train_augmented)

RandomForestClassifier(random_state=10)

In [29]:
# For XGBoost Model
# Make predictions on training and test sets
xgb_train_predictions = xgb_clf.predict(X_train_augmented)
xgb_test_predictions = xgb_clf.predict(X_test_reduced)

# Calculate accuracy
xgb_train_accuracy = accuracy_score(y_train_augmented, xgb_train_predictions)
xgb_test_accuracy = accuracy_score(y_test, xgb_test_predictions)

# Calculate misclassification rate
xgb_train_misclassification = 1 - xgb_train_accuracy
xgb_test_misclassification = 1 - xgb_test_accuracy

# For Random Forest Model
# Make predictions on training and test sets
rf_train_predictions = rf_clf.predict(X_train_augmented)
rf_test_predictions = rf_clf.predict(X_test_reduced)

# Calculate accuracy
rf_train_accuracy = accuracy_score(y_train_augmented, rf_train_predictions)
rf_test_accuracy = accuracy_score(y_test, rf_test_predictions)

# Calculate misclassification rate
rf_train_misclassification = 1 - rf_train_accuracy
rf_test_misclassification = 1 - rf_test_accuracy

# Print results
print("XGBoost Training Accuracy:", xgb_train_accuracy)
print("XGBoost Test Accuracy:", xgb_test_accuracy)
print("XGBoost Training Misclassification Rate:", xgb_train_misclassification)
print("XGBoost Test Misclassification Rate:", xgb_test_misclassification)

print("Random Forest Training Accuracy:", rf_train_accuracy)
print("Random Forest Test Accuracy:", rf_test_accuracy)
print("Random Forest Training Misclassification Rate:", rf_train_misclassification)
print("Random Forest Test Misclassification Rate:", rf_test_misclassification)

# Calculate the standard error of the misclassification rate for XGBoost
xgb_train_se = np.sqrt((xgb_train_misclassification * (1 - xgb_train_misclassification)) / len(X_train))
xgb_test_se = np.sqrt((xgb_test_misclassification * (1 - xgb_test_misclassification)) / len(X_test))

# Calculate the standard error of the misclassification rate for Random Forest
rf_train_se = np.sqrt((rf_train_misclassification * (1 - rf_train_misclassification)) / len(X_train))
rf_test_se = np.sqrt((rf_test_misclassification * (1 - rf_test_misclassification)) / len(X_test))

# Print the standard errors
print("XGBoost Training Misclassification SE:", xgb_train_se)
print("XGBoost Test Misclassification SE:", xgb_test_se)
print("Random Forest Training Misclassification SE:", rf_train_se)
print("Random Forest Test Misclassification SE:", rf_test_se)

XGBoost Training Accuracy: 1.0
XGBoost Test Accuracy: 0.6015625
XGBoost Training Misclassification Rate: 0.0
XGBoost Test Misclassification Rate: 0.3984375
Random Forest Training Accuracy: 1.0
Random Forest Test Accuracy: 0.5859375
Random Forest Training Misclassification Rate: 0.0
Random Forest Test Misclassification Rate: 0.4140625
XGBoost Training Misclassification SE: 0.0
XGBoost Test Misclassification SE: 0.04327284968965728
Random Forest Training Misclassification SE: 0.0
Random Forest Test Misclassification SE: 0.043536510010075705
